In [5]:
import pandas as pd
import os

# Reading data frome excel
excel_file_path = os.path.join(os.path.dirname(os.getcwd()), 'doc', 'case_study_data.xlsx')

# Reading each sheet into a separate DataFrame
df_facilities = pd.read_excel(excel_file_path, sheet_name='Facilities')
df_prov_fac_claims = pd.read_excel(excel_file_path, sheet_name='Provider-Facility Affiliated Cl')
df_prov_demog = pd.read_excel(excel_file_path, sheet_name='Provider Demographics')

# Displaying the first few rows of each DataFrame
print("Facilities:\n", df_facilities.head())
print("\nProvider-Facility Affiliated CL:\n", df_prov_fac_claims.head())
print("\nProvider Demographics:\n", df_prov_demog.head())


Facilities:
         Unnamed: 0        Unnamed: 1          Unnamed: 2  \
0    enrollment_id  enrollment_state  provider_type_code   
1  O20051101000429                OH               00-09   
2  O20051115000044                OH               00-09   
3  O20070216000074                OH               00-09   
4  O20070806000006                OH               00-09   

                   Unnamed: 3  Unnamed: 4    Unnamed: 5 Unnamed: 6  \
0          provider_type_text         npi  mulitple_npi        ccn   
1  PART A PROVIDER - HOSPITAL  1194763045             N     360087   
2  PART A PROVIDER - HOSPITAL  1215989611             N     360077   
3  PART A PROVIDER - HOSPITAL  1043397292             Y     369807   
4  PART A PROVIDER - HOSPITAL  1700828852             Y     360059   

     Unnamed: 7                                     Unnamed: 8  \
0  associate_id                              organization_name   
1    2466358387                              LUTHERAN HOSPITAL   
2    46

In [7]:
# Function to summarize data quality
def check_data_quality(df, df_name):
    print(f"Data Quality Report for: {df_name}\n")

    # missing values
    missing_values = df.isnull().sum()
    print("Missing Values:\n", missing_values[missing_values > 0], "\n")

    # duplicates
    duplicate_rows = df.duplicated().sum()
    print(f"Number of duplicate rows: {duplicate_rows}\n")

    # data types
    print("Data Types:\n", df.dtypes, "\n")

    # shape of numeric columns
    if df.select_dtypes(include=[float, int]).shape[1] > 0:
        print("Statistics for Numeric Columns:\n", df.describe(), "\n")

    print("-------------------------------------------")

# Check data quality for each DataFrame
check_data_quality(df_facilities, "Facilities")
check_data_quality(df_prov_fac_claims, "Provider-Facility Affiliated Claims")
check_data_quality(df_prov_demog, "Provider Demographics")


Data Quality Report for: Facilities

Missing Values:
 Unnamed: 9      7
Unnamed: 10     2
Unnamed: 11     2
Unnamed: 13     8
Unnamed: 16    10
Unnamed: 21    10
Unnamed: 35    10
Unnamed: 37    11
Unnamed: 38    11
dtype: int64 

Number of duplicate rows: 0

Data Types:
 Unnamed: 0     object
Unnamed: 1     object
Unnamed: 2     object
Unnamed: 3     object
Unnamed: 4     object
Unnamed: 5     object
Unnamed: 6     object
Unnamed: 7     object
Unnamed: 8     object
Unnamed: 9     object
Unnamed: 10    object
Unnamed: 11    object
Unnamed: 12    object
Unnamed: 13    object
Unnamed: 14    object
Unnamed: 15    object
Unnamed: 16    object
Unnamed: 17    object
Unnamed: 18    object
Unnamed: 19    object
Unnamed: 20    object
Unnamed: 21    object
Unnamed: 22    object
Unnamed: 23    object
Unnamed: 24    object
Unnamed: 25    object
Unnamed: 26    object
Unnamed: 27    object
Unnamed: 28    object
Unnamed: 29    object
Unnamed: 30    object
Unnamed: 31    object
Unnamed: 32    object
U